# Approach C Demo: Fine-tuned DeBERTa for NLI

## Imports

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Constants

In [3]:
FT_MODEL_PATH = './drive/MyDrive/Colab/NLU_CW/deberta-finetuned-nli'
TEST_PATH = './drive/MyDrive/Colab/NLU_CW/test.csv'

## Load Fine-tuned Model

In [4]:
ft_model_tokenizer = AutoTokenizer.from_pretrained(f"{FT_MODEL_PATH}")
ft_model = AutoModelForSequenceClassification.from_pretrained(f"{FT_MODEL_PATH}")

In [5]:
ft_model.to(device)

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine

## Test Dataset

In [6]:
test_df = pd.read_csv(TEST_PATH)
test_df.head(10)

,premise,hypothesis
0,"Boy wearing red hat, blue jacket pushing plow ...",The boy is surrounded by snow
1,A blond woman in a black shirt is standing beh...,The woman is standing.
2,Three people in uniform are outdoors and are o...,Uniformed people are outside
3,"A person, in a striped blue shirt and pants, i...",The person is running
4,"A man, woman, and child get their picture take...",A family on vacation is posing.
5,A tennis player in blue shorts and a white shi...,The person is taking a nap in their bed.
6,A boy looks on at an electric device with thre...,A boy is taking a nap in his bed.
7,A young girl is standing in a kitchen holding ...,A boy is playing with a firetruck.
8,Three men in yellow green safety vests sit on ...,People are playing volleyball on the sand.
9,man dressed in orange clothing with face cover...,Two men are in a shopping mall.


## Generate Predictions

In [7]:
predictions = []

for index, row in test_df.iterrows():
  premise = row['premise']
  hypothesis = row['hypothesis']

  input = ft_model_tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
  output = ft_model(input["input_ids"].to(device))

  prediction = torch.softmax(output["logits"][0], -1).tolist()
  if(prediction[0] < prediction[1]):
    predictions.append(1)
  else:
    predictions.append(0)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
test_df['prediction'] = predictions

In [9]:
test_df

,premise,hypothesis,prediction
0,"Boy wearing red hat, blue jacket pushing plow ...",The boy is surrounded by snow,1
1,A blond woman in a black shirt is standing beh...,The woman is standing.,1
2,Three people in uniform are outdoors and are o...,Uniformed people are outside,1
3,"A person, in a striped blue shirt and pants, i...",The person is running,1
4,"A man, woman, and child get their picture take...",A family on vacation is posing.,1
...,...,...,...
3297,A younger man dressed in tribal attire.,A young man wears clothing.,1
3298,A woman within an orchestra is playing a violin.,A man is looking in a telescope.,1
3299,You can see the bottom halves of three boys st...,There are three boys standing.,1
3300,A crowd of people fill the street in front of ...,The crowd is playing hockey.,0


## Save Results

In [14]:
test_df['prediction'].to_csv('Group_2_C.csv', index=False)